In [1]:
# Import packages
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.dataset import Dataset
from sklearn.externals import joblib
from azureml.core import Datastore


In [2]:
# Connect to workspace
ws = Workspace.from_config()

In [3]:
# Get model
model_path = Model.get_model_path(model_name = "AutoMLf73e1768422", version=None, _workspace=ws)
model = joblib.load(model_path)

In [4]:
# Get test data
test_dataset = Dataset.get_by_name(ws, name='credit_default_test_data')
test_df = test_dataset.to_pandas_dataframe()

In [5]:
pred_results = model.predict(test_df.drop(['Column1', 'default payment next month'], axis=1))
results = test_df.drop(['Column1'], axis=1)
results['PREDICTION'] = pred_results
results.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,PREDICTION
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,689,0,0,0,0,1,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3455,3261,0,1000,1000,1000,0,2000,1,0
2,90000,2,2,2,34,0,0,0,0,0,...,14948,15549,1518,1500,1000,1000,1000,5000,0,0
3,50000,2,2,1,37,0,0,0,0,0,...,28959,29547,2000,2019,1200,1100,1069,1000,0,0
4,50000,1,1,2,37,0,0,0,0,0,...,19619,20024,2500,1815,657,1000,1000,800,0,0


In [6]:
# Upload training and test data to blob store
local_path = 'data/credit_default_results_data.csv'
results.to_csv(local_path)

# get the datastore to upload prepared data
datastore = Datastore(ws, 'blobstorage')

# upload the local file from src_dir to the target_path in datastore
datastore.upload_files([local_path], target_path='results')

Uploading an estimated of 1 files
Target already exists. Skipping upload for results/credit_default_results_data.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_938fa9a141a04ff5b6f9ce0173d1e436